# **INVENTORY MANAGEMENT SYSTEM**

VERSION: 3

In [2]:
# INVENTORY MANAGEMENT SYSTEM

# Importing necessary libraries

import os
import json


# Json file to store inventory(Just a text file format looks like dictionery)
FILE_NAME = "inventory.json"

#----------------------------------------------
# LOAD & SAVE FUNCTIONS
#----------------------------------------------


#----------------------------------------------
# LOAD INVENTORY
#----------------------------------------------

# Define a function to load inventory

def load_inventory():                     # Load inventory from json file or returns empty dict if no file found
  if os.path.exists(FILE_NAME):           # Check the inventory.json file exists, if yes means we have saved inventory earlier
    with open(FILE_NAME, "r") as file:    # Open file in read mode
      try:
        return json.load(file)              # Reads json file and convert back into python's dictionery
      except json.JSONDecodeError:
        return {}  # If file is empty or corrupted, start fresh
  return {}                               # Returns empty dictionery if no file found


#----------------------------------------------
# SAVE INVENTORY
#----------------------------------------------

# Define a function to save current inventory to Json file

def save_inventory():                     # Save current inventory to json file
  with open(FILE_NAME, "w") as file:      #   Open file in write mode
    json.dump(inventory, file, indent = 4)  # Takes python dictionery and writes it to JSON file (indent-4 for easy readability)



#----------------------------------------------
# VIEW INVENTORY
#----------------------------------------------


# Our inventory will be stored as dictionery

# Define a function to View inventory

def view_inventory():    # Display current inventory

  if not inventory:                # Not inventory print empty
    print("Inventory is Empty!")
    return

  print("\n------- Current Inventory -------")   # Print header
  print("{:<10} {:<15} {:<10} {:<10}".format("Item_ID", "Name", "Quantity","Price"))  # Print column titles with spacing between the columns
  print("-"*50)       # Printing line of dashes

  for item_id, details in inventory.items():   # Loop over every item in inventory
    print("{:<10} {:<15} {:<10} {:<10}".format(
        item_id, details['name'], details['quantity'], details['price']
    ))       # Item_id-->key, details---> dictionery. For each item id print its id, name, quantity and price in columns

    print("-"*50)


#----------------------------------------------
# ADD ITEMS
#----------------------------------------------

# Define a function to add item
def add_item():

  # Getting the input from user
  item_id = input("Enter the item ID(Alphanumeric, Eg:A101): ").strip().upper()

  if item_id in inventory:
    print("Item already exists!")     #Exit function if item already exists
    return

  name = input("Enter the item name: ").strip().title()  # Ensure without spaces and starting in caps

  if not name:        # Checking an entry
    print("Item name can't be empty")
    return

  try:
    quantity = int(input("Enter the quantity: "))
    price = float(input("Enter the Unit price: "))

  except ValueError:
    print("Invalid input!, Quantity must be an integer, Price must be a number")
    return

  if quantity < 0 or price < 0:    # Checking positive number
    print("Quantity and price must be non negative")
    return

  inventory[item_id] = {"name": name, "quantity": quantity, "price": price}

  save_inventory()

  print(f"Item {name} added successfully!")

  view_inventory()


#----------------------------------------------
# UPDATE ITEMS
#----------------------------------------------

# Define a function to update item

def update_item():

  item_id = input("Enter the item_id to update: ").strip().upper() # getting item id to update

  if item_id not in inventory:
    print("Item not found!")   # Exit function if item_id not found
    return

  details = inventory[item_id]
  print(f"Current Inventory details: {details}") # print available items

  # Name update
  name = input("Enter the new name(leave blank to keep the same name): ").strip()

  if name:
    details["name"] = name.title()   # For title case


  # Quantity update with add/replace option

  quantity = input("Enter the new quantity(leave blank to keep the same quantity): ").strip()

  if quantity:

    try:
      quantity = int(quantity)   # Getting integer input
      choice = input("Do you want to replace(R) or add(A) to existing quantity?").strip().upper()

      if choice == 'A':     # If add qty to existing qty
        details["quantity"] += quantity
      else:
        details["quantity"] = quantity

    except ValueError:
      print("Invalid input update, Skipping quantity update!")

  # Price update
  price = input("Enter the new price(leave blank to keep the same price): ").strip()

  if price:
    try:
      details["price"] = float(price)
    except ValueError:
      print("Invalid price, Skipping price update!")


  inventory[item_id] = details

  save_inventory()

  print("Item updated successfully! Updated details:")
  print(inventory[item_id])

  load_inventory()



#----------------------------------------------
# DELETE ITEMS
#----------------------------------------------


# Define a function to delete item

def delete_item():

  keyword = input("Enter item ID or Name to delete: ").strip().lower()

  # Match by ID or Name
  item_id = None
  for iid, details in inventory.items():
      if keyword == iid.lower() or keyword == details["name"].lower():
          item_id = iid
          break

  if not item_id:
      print("Item not found!")
      return

  details = inventory[item_id]
  print(f"Current details: {details}")

  choice = input("Do you want to delete Entire item (E) or Quantity (Q)? ").strip().upper()

  if choice == 'E':
      del inventory[item_id]
      print("Item deleted successfully!")
  elif choice == 'Q':
      try:
          qty_to_delete = int(input("Enter the quantity to delete: "))
          if qty_to_delete <= 0:
              print("Quantity must be positive!")
              return
          if qty_to_delete >= details['quantity']:
              del inventory[item_id]
              print("All quantity removed. Item deleted from inventory.")
          else:
              details["quantity"] -= qty_to_delete
              inventory[item_id] = details
              print(f"{qty_to_delete} units removed. Updated quantity: {details['quantity']}")
      except ValueError:
          print("Invalid quantity input")
          return
  else:
      print("Invalid choice!")


#----------------------------------------------
# SEARCH ITEMS
#----------------------------------------------


# Define a function to search item

def search_item():

  keyword = input("Enter Item_ID or Name to search: ").strip().lower()  # Ask to search and convert to lower case to ignore upper and lower

  matches = []                                 # Collect all matching items

  for item_id, details in inventory.items():
        # Check match by exact ID or keyword inside the name (case-insensitive)
        if keyword == item_id.lower() or keyword in details['name'].lower():
            matches.append((item_id, details))

  print("\n------- Search Results -------")   # Header

  if matches:
    for item_id, details in matches:
            print(f" Found → ID: {item_id}, Name: {details['name']}, Qty: {details['quantity']}, Price: {details['price']}")

  else:   # If nothing matched, prints no matching

      print("No matching item found!")


# Define a function to for low stock alert

def low_stock_alert(threshold=5):   # If no threshold is given, it takes 5 as default

  print("\n------- Low Stock Items -------")   # Header

  found = False   # Flag to found any low stock items exist

  for item_id,details in inventory.items():  # Loop over each items
    if details['quantity'] < threshold:    # If it less than threshold(5)
      print(f" {details['name']} (ID: {item_id}) - only {details['quantity']} left!")  # Print warning howmany left

      found = True     # Found atlease one item

  if not found:    # If not found

      print("All items have sufficient stock!")  # Print sufficient stocks



#----------------------------------------------------------------------------

# USER PURCHASE SYSTEM

#----------------------------------------------------------------------------

#----------------------------------------------
# SHOW DATABASE
#----------------------------------------------


def show_DB():

  if not inventory:
        print("Database is empty!")
        return
  print("\n======= DATABASE CONTENT =======")
  print(f"{'Item_ID':<10}{'Name':<15}{'Quantity':<10}{'Price':<10}")
  print("-" * 50)
  for item_id, details in inventory.items():
      print(f"{item_id:<10}{details['name']:<15}{details['quantity']:<10}{details['price']:<10}")
  print("-" * 50)


#----------------------------------------------
# PURCHASE ITEMS
#----------------------------------------------

def purchase_items():
  if not inventory:
        print("Inventory is Empty!, Nothing to purchase.")
        return

  cart = {}

  while True:
        view_inventory()
        item_id = input("Enter item_id to purchase or 'done' to finish: ").strip().upper()

        if item_id == 'DONE':
            break
        if item_id not in inventory:
            print("Invalid Item_id! Please try again!")
            continue

        try:
          qty = int(input(f"Enter quantity for {inventory[item_id]['name']}: "))

          if qty <= 0:
            print("Quantity must be greater than 0")
            continue
          if qty > inventory[item_id]["quantity"]:
            print(f" Sorry! only {inventory[item_id]['quantity']} units available!")
            continue
        except ValueError:
          print("Invalid quanity. Please enter a number")
          continue


        # Add to cart
        if item_id in cart:
          cart[item_id]['quantity'] += qty
        else:
          cart[item_id] = {
              "name" : inventory[item_id]["name"],
              "price" : inventory[item_id]["price"],
              "quantity" : qty

          }

        # reduced from stock

        inventory[item_id]["quantity"] -= qty

        save_inventory()

        print(f"Added {qty} x {inventory[item_id]['name']} to cart.")


  # Print Bill
  if not cart:
    print("No items purchased!")
    return

  print("\n========== FINAL BILL ==========")
  print("{:<10} {:<15} {:<10} {:<10}".format("Item_ID",'Name','Qty',"Subtotal"))
  print("-"*50)


  total = 0

  for item_id, details in cart.items():
    Subtotal = details["quantity"] * details['price']
    total += Subtotal

    print("{:<10} {:<15} {:<10} {:<10}".format(item_id, details["name"], details['quantity'], Subtotal))

  print("-"*50)
  print(f" TOTAL AMOUNT: {total}")
  print("Thank you shopping with us!")



#-------------------------

# Main Menu Loop

#-------------------------

# Define main menu function to select choices and react

def menu():

  while True:   # Keep running until we choose to exit

    print("\n======== INVENTORY MANAGEMENT SYSTEM =======")   # Header
    print("1. Add item")
    print("2. Update Item")
    print("3. Delete Item")
    print("4. View Inventory")
    print("5. Search Item")
    print("6. Low Stock Alert")
    print("7. Show DataBase")
    print("8. Purchase items")
    print("9. Exit the Program")


    choice = input("Enter your choice(1-9): ").strip()   # Getting input to choose what they want

    if choice == "1":   # If 1 call add_item
      add_item()
    elif choice == "2":
      update_item()
    elif choice == "3":
      delete_item()
    elif choice == "4":
      view_inventory()
    elif choice == "5":
      search_item()
    elif choice == "6":
      low_stock_alert()
    elif choice == '7':
        show_DB()
    elif choice == '8':
      purchase_items()
    elif choice == "9":
      print("Exiting...Inventory saved. Good bye!")
      break    # Break the loop if user choose to exit the program
    else:
      print("Invalid choice, Please try again!")   # If user enter nothing, prints and loop restarts


# Run the program
inventory = load_inventory()
menu()      # Calls the main menu and starts the whole program



======== INVENTORY MANAGEMENT SYSTEM =======
1. Add item
2. Update Item
3. Delete Item
4. View Inventory
5. Search Item
6. Low Stock Alert
7. Show DataBase
8. Purchase items
9. Exit the Program
Enter your choice(1-9): 1
Enter the item ID(Alphanumeric, Eg:A101): A101
Enter the item name: APPLE
Enter the quantity: 10
Enter the Unit price: 25
Item Apple added successfully!

------- Current Inventory -------
Item_ID    Name            Quantity   Price     
--------------------------------------------------
A101       Apple           10         25.0      
--------------------------------------------------

======== INVENTORY MANAGEMENT SYSTEM =======
1. Add item
2. Update Item
3. Delete Item
4. View Inventory
5. Search Item
6. Low Stock Alert
7. Show DataBase
8. Purchase items
9. Exit the Program
Enter your choice(1-9): 1
Enter the item ID(Alphanumeric, Eg:A101): A18
Enter the item name: avocado
Enter the quantity: 15
Enter the Unit price: 50
Item Avocado added successfully!

------- Curre